In [1]:
import numpy as np
import xarray as xr
import os
import pandas as pd
import glob

In [2]:
#import sys ; sys.path.append("../..")
from ppe_tools import *

In [3]:
# set directories
exp_dir = '/glade/work/linnia/CLM6-PPE/tmp/'

### 1. Define Parameters and Ranges

In [10]:
# file with parameter ranges
csv='ctsm6sp_lhc_paramranges_11192024.csv'

In [12]:
df = pd.read_csv(csv)
params=df['param'].values
params

array(['taulnir', 'taulvis', 'tausnir', 'tausvis', 'rholnir', 'rholvis',
       'rhosnir', 'rhosvis', 'xl', 'displar', 'dleaf', 'z0mr', 'csoilc',
       'cv', 'a_coef', 'a_exp', 'zlnd', 'zsno', 'd_max',
       'frac_sat_soil_dsl_init', 'lai_dl', 'z_dl', 'zetamaxstable',
       'wind_min', 'tkd_sand', 'tkd_clay', 'tkd_om', 'tkm_om', 'pd',
       'csol_om', 'csol_sand', 'csol_clay', 'bsw_sf', 'hksat_sf',
       'sucsat_sf', 'watsat_sf', 'sand_pf', 'clay_pf', 'baseflow_scalar',
       'maximum_leaf_wetted_fraction', 'interception_fraction',
       'aq_sp_yield_min', 'fff', 'liq_canopy_storage_scalar',
       'snow_canopy_storage_scalar', 'perched_baseflow_scalar', 'e_ice',
       'n_baseflow', 'n_melt_coef', 'accum_factor', 'eta0_anderson',
       'eta0_vionnet', 'drift_gs', 'ssi', 'wimp',
       'upplim_destruct_metamorph', 'wind_snowcompact_fact', 'rho_max',
       'tau_ref', 'snowcan_unload_wind_fact', 'snowcan_unload_temp_fact',
       'xdrdt', 'snw_rds_refrz', 'scvng_fct_mlt_sf', 'ce

In [13]:
kcns=['kc_nonmyc',
        'kn_nonmyc',
        'akc_active',
        'akn_active',
        'ekc_active',
        'ekn_active']

### 2. Build Parameter Dictionary

In [14]:
### we should maybe make this a function?
lhcs = {}
for p in params:
    ix       = df['param']==p
    minval   = df['min'][ix].values[0]
    maxval   = df['max'][ix].values[0]
    pftmin   = df['pft_mins'][ix].values[0]
    pftmax   = df['pft_maxs'][ix].values[0]
    thisloc  = df['loc'][ix].values[0]
    if p=='KCN':
        flag='KCN'
    else:
        flag=''
    
    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    if p!='KCN':
        lhcs[p]={'min':thismin,'max':thismax,'loc':thisloc,'flagged':[]}
    else:
        flagged={}
        for kcn in kcns:
            flagged[kcn]={'min':thismin,'max':thismax,'loc':thisloc}
        lhcs[p]={'min':[],'max':[],'loc':thisloc,'flagged':flagged}

### 3. Define Ensemble

##### a. If you already know the parameter sets to run, read them in here
##### b. If you would like to generate a new Latin-Hypercube sample, skip to step 4

In [10]:
# specify where your samples exist
# make sure that they are in the same order as the params in lhcs above
psets = pd.read_csv('~/clm5ppe/pyth/exp1_EmBE/exp1_EmBE_BiomeSpecific_psets.csv',index_col=0)
nens = 24 # how many ensemble members do you have ? 

In [11]:
def get_p(p,i):
    return psets.iloc[i].loc[p]

In [12]:
exp1=[[get_p(p,i) for p in lhcs] for i in range(nens)]

In [13]:
#Rows are ensemble members and columns are parameters (scale factors)
np.shape(exp1)

(24, 32)

### 4. Create the Ensemble Object

In [16]:
# 4a. instantiate the Ensemble object
#basefile = '/glade/campaign/asp/djk2120/PPEn11/paramfiles/OAAT0000.nc'
basefile = '/glade/work/linnia/ctsm5.3.010/cime/scripts/transient/runtime_files/ctsm60_params.c241017.nc'
pdir = exp_dir + 'paramfiles/'
ndir = exp_dir + 'namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

# 4b. add the new ensemble members
prefix = 'exp1'   #make your own prefix
nextnum=1
n_samples=40      # set the number of ensemble members to generate
x.add_lhcs(lhcs,prefix,nextnum,n_samples,lhc=None) # for new LHC sample set lhc=None

### 5. Write the param_files

In [17]:
paramset_file = exp_dir + 'psets_tmp.txt'
x.write(lhcfile=paramset_file)

In [18]:
%%bash -s "$paramset_file"
# Check that the file was created
paramset_file=$1 

stat $paramset_file

  File: /glade/work/linnia/CLM6-PPE/tmp/psets_tmp.txt
  Size: 165354    	Blocks: 352        IO Block: 8388608 regular file
Device: 41h/65d	Inode: 219527543   Links: 1
Access: (0644/-rw-r--r--)  Uid: (35240/  linnia)   Gid: ( 1000/    ncar)
Access: 2024-11-19 11:56:13.282776000 -0700
Modify: 2024-11-19 11:56:13.297637663 -0700
Change: 2024-11-19 11:56:13.297637663 -0700
 Birth: -
